In [14]:
!pip install -q transformers accelerate


In [15]:
from google.colab import files

print(" Please upload your `chapter_1_raw.txt` file:")
uploaded = files.upload()

filename = list(uploaded.keys())[0]

with open(filename, "r", encoding="utf-8") as f:
    original_text = f.read()

print(" File uploaded and loaded successfully.")


 Please upload your `chapter_1_raw.txt` file:


Saving chapter_1_raw.txt to chapter_1_raw (8).txt
 File uploaded and loaded successfully.


In [ ]:
from transformers import pipeline

# Load the local AI model from HuggingFace
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

# Prompt to rewrite the content
prompt = f"Rewrite the following passage in fluent, modern English:\n\n{original_text}"

# Generate the rewritten output
output = generator(prompt, max_length=1024, do_sample=True, temperature=0.7)

# Extract the rewritten text
rewritten = output[0]['generated_text']
print("Rewritten Chapter Preview:\n\n", rewritten[:1000])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Rewritten Chapter Preview:

 Rewrite the following passage in fluent, modern English:

"The Gates of Morning"

CHAPTER I

THE CANOE BUILDER

DICK standing on a ledge of coral cast his eyes to the South.

Behind him the breakers of the outer sea thundered and the spindrift scattered on the wind; before him stretched an ocean calm as a lake, infinite, blue, and flown about by the fishing gulls—the lagoon of Karolin.

Clipped by its forty-mile ring of coral this great pond was a sea in itself, a sea of storm in heavy winds, a lake of azure, in light airs—and it was his—he who had landed here only yesterday.

Women, children, youths, all the tribe to be seen busy along the beach in the blazing sun, fishing with nets, playing their games or working on the paraka patches, all were his people. His were the canoes drawn up on the sand and his the empty houses where the war canoes had once rested on their rollers.

Then as he cast his eyes from the lagoon to the canoe houses his brow contracted

In [16]:
# Save to .txt file
with open("chapter_1_spun.txt", "w", encoding="utf-8") as f:
    f.write(rewritten)

# Download it
files.download("chapter_1_spun.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
# Show rewritten output and accept feedback
print("AI Rewritten Draft:\n\n", rewritten[:1000])

# Allow user to enter a small edit (simulate editor review)
editor_feedback = input("Enter any manual correction you'd like to apply:\n")

# Apply a small update
final_version = rewritten + "\n\n[Editor Note]: " + editor_feedback


AI Rewritten Draft:

 Rewrite the following passage in fluent, modern English:

"The Gates of Morning"

CHAPTER I

THE CANOE BUILDER

DICK standing on a ledge of coral cast his eyes to the South.

Behind him the breakers of the outer sea thundered and the spindrift scattered on the wind; before him stretched an ocean calm as a lake, infinite, blue, and flown about by the fishing gulls—the lagoon of Karolin.

Clipped by its forty-mile ring of coral this great pond was a sea in itself, a sea of storm in heavy winds, a lake of azure, in light airs—and it was his—he who had landed here only yesterday.

Women, children, youths, all the tribe to be seen busy along the beach in the blazing sun, fishing with nets, playing their games or working on the paraka patches, all were his people. His were the canoes drawn up on the sand and his the empty houses where the war canoes had once rested on their rollers.

Then as he cast his eyes from the lagoon to the canoe houses his brow contracted, and, 

In [18]:
# Save versioned copy
version = "v1"  # You can later change this to v2, final, etc.
versioned_filename = f"chapter_1_{version}.txt"

with open(versioned_filename, "w", encoding="utf-8") as f:
    f.write(final_version)

files.download(versioned_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
!pip install chromadb


In [20]:
import chromadb
from chromadb.utils import embedding_functions

# Create Chroma client
client = chromadb.Client()

#  Use get_or_create=True to avoid InternalError
collection = client.create_collection(
    name="chapter_versions",
    get_or_create=True
)

print("Collection ready.")





Collection ready.


In [21]:
# Add the final_version to the collection
collection.add(
    documents=[final_version],  # The text content
    metadatas=[{"version": version, "original_filename": filename}], # Metadata
    ids=["chapter_1_" + version]  # Unique ID for the document
)

print(f"Document '{'chapter_1_' + version}' added to collection.")

# Semantic search from ChromaDB
results = collection.query(
    query_texts=["fluent modern rewrite of a classic chapter"],
    n_results=1
)

# Add a check to see if any documents were returned
if results and results.get('documents') and results['documents'][0]:
    print(" Top match retrieved:")
    print(" Document:\n", results['documents'][0][0])
    print(" Metadata:", results['metadatas'][0][0])
else:
    print(" No matching documents found for the query.")

Document 'chapter_1_v1' added to collection.
 Top match retrieved:
 Document:
 Rewrite the following passage in fluent, modern English:

"The Gates of Morning"

CHAPTER I

THE CANOE BUILDER

DICK standing on a ledge of coral cast his eyes to the South.

Behind him the breakers of the outer sea thundered and the spindrift scattered on the wind; before him stretched an ocean calm as a lake, infinite, blue, and flown about by the fishing gulls—the lagoon of Karolin.

Clipped by its forty-mile ring of coral this great pond was a sea in itself, a sea of storm in heavy winds, a lake of azure, in light airs—and it was his—he who had landed here only yesterday.

Women, children, youths, all the tribe to be seen busy along the beach in the blazing sun, fishing with nets, playing their games or working on the paraka patches, all were his people. His were the canoes drawn up on the sand and his the empty houses where the war canoes had once rested on their rollers.

Then as he cast his eyes from 